In [1]:
import MDAnalysis as mda
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from MDAnalysis.lib.distances import distance_array

/tmp/ipykernel_959972/3919984661.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


## Loading data and creating atom group of ions

In [2]:
import MDAnalysis as mda
u = mda.Universe('md.gro', 'md.xtc')
print(u, '\n', u.atoms, '\n', u.trajectory)

ions= u.select_atoms('resname NA CL')
print(len(ions))

print(len(u.trajectory))

<Universe with 8223 atoms> 
 <AtomGroup [<Atom 1: OW of type O of resname SOL, resid 49 and segid SYSTEM>, <Atom 2: HW1 of type H of resname SOL, resid 49 and segid SYSTEM>, <Atom 3: HW2 of type H of resname SOL, resid 49 and segid SYSTEM>, ..., <Atom 8221: OW of type O of resname SOL, resid 2757 and segid SYSTEM>, <Atom 8222: HW1 of type H of resname SOL, resid 2757 and segid SYSTEM>, <Atom 8223: HW2 of type H of resname SOL, resid 2757 and segid SYSTEM>]> 
 <XTCReader md.xtc with 251 frames of 8223 atoms>
96
251


## Function for calculating osmotic values:

In [3]:
k_joules_per_nm2 = 1000 
k_joules_per_angstrom2 = k_joules_per_nm2/100 #force constant (k) 1000(kJ/mol/nm^2) to (kJ/mol/A^2)


def osmotic_values(mean_force_wall,molarity):
    cross_sectional_area=30*30 #A^2 
    osm_press=mean_force_wall/cross_sectional_area
    bar=osm_press*1000/(6.022e23*1e-27*101.32) #kJ/mol/A^3 to bar
    print(f"Osmotic Pressure:", "%.2f"%bar, "bar")
    R_idgas=8.314
    temp=300
    v_H2O=2
    osm_coeff=(bar/(v_H2O*molarity*R_idgas*temp))*101.32 #Osmotic pressure to osmotic coefficient
    print(f"Osmotic Coefficient:", "%.2f"%osm_coeff)


## Attempt #1

In [4]:
z_wall=[30,60] # location of wall in z direction in Angstroms
av_meanF=[]
for z_w in z_wall:
    z_diff = np.zeros(len(u.trajectory))

    for i,ts in enumerate(u.trajectory[1:]):
        z_diff[i]=np.abs(ions.positions[:,2] - z_w).sum()

    av_fwall=z_diff.mean()*k_joules_per_angstrom2
    print(f"Mean Force from wall at {z_w} A:", "%.2f"%av_fwall, "kJ mol^-1 A^-1")
    av_meanF.append(av_fwall)

total_av_meanF=np.average(av_meanF)
print(f"Total Mean Force from two walls:", "%.2f"%total_av_meanF, "kJ mol^-1 A^-1")

osmotic_values(total_av_meanF,0.98)

Mean Force from wall at 30 A: 15058.08 kJ mol^-1 A^-1
Mean Force from wall at 60 A: 13630.02 kJ mol^-1 A^-1
Total Mean Force from two walls: 14344.05 kJ mol^-1 A^-1
Osmotic Pressure: 261212.06 bar
Osmotic Coefficient: 5413.79


## Attempt #2

The Euclidean distance simplifies to the absolute value of the difference in the z-direction because the wall is parallel to the x-y plane and I am only interested in the z-coordinate differences

In [14]:
z_wall=[30,60] # location of wall in z direction in Angstroms
av_meanF=[]

for z_w in z_wall:
   
   av_dists = np.zeros(len(u.trajectory))

   for i, ts in enumerate(u.trajectory):
      z_diff= np.abs(ions.positions[:, 2] - z_w)  # Absolute difference between ion z-positions and z_wall for this frame
      av_dists[i] = z_diff.mean()    # Average distance for this frame

   # Calculating the overall average distance across all frames multiplied by k (force constant)
   av_fwall = k_joules_per_angstrom2*av_dists.mean()
   print(f"Mean Euclidean Distance to wall at {z_w} Å:", "%.2f"%av_fwall, "Å")
   av_meanF.append(av_fwall)

total_av_meanF=np.average(av_meanF)
print(f"Total Mean Force from two walls:", "%.2f"%total_av_meanF, "kJ mol^-1 A^-1")

osmotic_values(total_av_meanF,0.98)

Mean Euclidean Distance to wall at 30 Å: 157.48 Å
Mean Euclidean Distance to wall at 60 Å: 142.55 Å
Total Mean Force from two walls: 150.01 kJ mol^-1 A^-1
Osmotic Pressure: 2731.84 bar
Osmotic Coefficient: 56.62


Without multiplying by k constant:

In [17]:
z_wall=[30,60] # location of wall in z direction in Angstroms
av_meanF=[]

for z_w in z_wall:
   
   av_dists = np.zeros(len(u.trajectory))

   for i, ts in enumerate(u.trajectory):
      z_diff= np.abs(ions.positions[:, 2] - z_w)  # Absolute difference between ion z-positions and z_wall for this frame
      av_dists[i] = z_diff.mean()    # Average distance for this frame

   # Calculating the overall average distance across all frames 
   av_fwall = av_dists.mean()
   print(f"Mean Euclidean Distance to wall at {z_w} Å:", "%.2f"%av_fwall, "Å")
   av_meanF.append(av_fwall)

total_av_meanF=np.average(av_meanF)
print(f"Total Mean Force from two walls:", "%.2f"%total_av_meanF, "kJ mol^-1 A^-1")

osmotic_values(total_av_meanF,0.98)

Mean Euclidean Distance to wall at 30 Å: 15.75 Å
Mean Euclidean Distance to wall at 60 Å: 14.26 Å
Total Mean Force from two walls: 15.00 kJ mol^-1 A^-1
Osmotic Pressure: 273.18 bar
Osmotic Coefficient: 5.66
